
# Assignment 2

* For Computational Economics at Waseda
* Instructor: John Stachurski 
* Due: Friday 21st Nov
* Submission:
    * to john.stachurski@gmail.com
    * subject: assignment 2
    * filename: firstname_lastname_studentnumber_ass_2.ipynb

Remember to restart the session and run one last time before you download and submit.

## Exercise 1  

If $Y$ is a random payoff and an agents evaluation of the payoff is $e := \mathbb{E} Y$,
then we say that the agent is **risk neutral**.

Sometimes we want to model agents as risk averse.  One way to do this is to
change their evaluation to

$$
    e_{\theta} = \frac{1}{\theta} \ln\left( \mathbb{E} [ \exp(\theta Y) ] \right)
$$

where $\theta$ is a number satisfying $\theta < 0$.

Suppose that $Y$ has the normal distribution $N(\mu, \sigma^2)$, so that its
mean is $\mu$ and its variance is $\sigma^2$.

Using pencil and paper, write down an expression for $e_\theta$.

Fix $\theta = -1$ and produce a contour plot of $e_\theta$ as a function of
$\mu$ and $\sigma$.   

Does this agent like a higher payoff on average?

Does she like more risk?



## Exercise 2  

Suppose now that $Y$ has the Beta$(a, b)$ distribution:
https://scipy.github.io/devdocs/reference/generated/scipy.stats.beta.html#scipy.stats.beta

Set $a=b=2.0$ and calculate

$$
    e_{\theta} = \frac{1}{\theta} \ln\left( \mathbb{E} [ \exp(\theta Y) ] \right)
$$

using Monte Carlo 

* sample $Y_1, \ldots, Y_n$ from Beta$(2,2)$ 
* replace $\mathbb{E}$ with an average over $\exp(\theta Y_i)$

Do this for $\theta$ in a grid of 100 points between $-2$ and $-0.1$.

Use a large number of Monte Carlo samples for each $\theta$.

Plot your estimate of $e_{\theta}$ against $\theta$.  Be sure to label your graph.

## Exercise 3  

Keep $Y$ as in the previous exercise and fix $\theta = -2$.

Using Monte Carlo again, calculate

$$
    e_{\theta} = \frac{1}{\theta} \ln\left( \mathbb{E} [ \exp(\theta X) ] \right)
$$

where $X = Y + \sigma Z$ and $Z$ is standard normal.

How does $e_\theta$ change with $\sigma$?

Can you provide some intuition for what is happening (given that the agent is
risk averse)?

Use a plot to illustrate your results.

## Exercise 4

In the QuantEcon lectures we studied a job search model with

* separation
* Markov wage draws and
* fitted value function iteration.

The wage offer process is continuous and obeys

$$
    W_t = \exp(X_t)
    \quad \text{where} \quad
    X_{t+1} = \rho X_t + \nu Z_{t+1}
$$

and $\{Z_t\}$ is IID and standard normal.

We can also write this in terms of wages by taking exponentials of the second
equation, to get

$$
    W_{t+1} = W_t^\rho  \exp(\nu Z_{t+1})
$$

The unemployed worker's value function satisfied the Bellman equation

$$
    v_u(w) = \max\{v_e(w), c + \beta (P v_u)(w)\}
$$

The employed worker's value function satisfies the Bellman equation

$$
    v_e(w) = 
        w + \beta
        \left[
            \alpha (P v_u)(w) + (1-\alpha) v_e(w)
        \right]
$$

(Here we assuming that $u(x) = x$, so that utility is just earnings.)

In each case, 

$$
    (P v_u)(w) := \int v_u(w') p(w, w') d w'
$$

where $p(w, \cdot)$ is the conditional density of $w'$ given $w$.

This is the expected value of being unemployed next period, given current wage
$w$.

We can write this more explicitly as

$$
    (P v_u)(w) := \mathbb{E} v_u( w^\rho  \exp(\nu Z) )
$$

Here 

* $Z$ is a standard normal random variable
* $w$ is a constant -- today's wage, which we take as given.

Alternatively, we can write

$$
    (P v_u)(w) := \int v_u( w^\rho  \exp(\nu z) ) \psi(z) dz,
$$

where $\psi$ is the standard normal density.

We understand $v_u$ and $v_e$ as functions on all of $\mathbb{R}_+$.

We solved the model using a version of value function iteration and studied the
reservation wage.

Your task is to study the same model but replacing the assumption of risk
neutral expectations with risk averse expectations.

In particular,

$$
    (P v_u)(w) = \mathbb{E} v_u( w^\rho  \exp(\nu Z) )
$$

is replaced with

$$
    (P_\theta v_u)(w)
    = \frac{1}{\theta} \ln
        \left[
            \mathbb{E} \exp(\theta v_u( w^\rho  \exp(\nu Z) ))
        \right]
$$

Otherwise the model is the same.

Use the default parameters below.


In [ ]:
import jax
import jax.numpy as jnp
from jax import lax
import quantecon as qe
from typing import NamedTuple

class Model(NamedTuple):
    c: float              # unemployment compensation
    α: float              # job separation rate
    β: float              # discount factor
    ρ: float              # wage persistence
    ν: float              # wage volatility
    θ: float              # risk aversion parameter
    w_grid: jnp.ndarray   # grid of points for fitted VFI
    z_draws: jnp.ndarray  # draws from the standard normal distribution

def create_mccall_model(
        c: float = 1.0,
        α: float = 0.1,
        β: float = 0.96,
        ρ: float = 0.9,
        ν: float = 0.2,
        θ: float = -1.5,
        grid_size: int = 100,
        mc_size: int = 1000,
        seed: int = 1234
    ):
    """Factory function to create a McCall model instance."""

    key = jax.random.PRNGKey(seed)
    z_draws = jax.random.normal(key, (mc_size,))

    # Discretize just to get a suitable wage grid for interpolation
    mc = qe.markov.tauchen(grid_size, ρ, ν)
    w_grid = jnp.exp(jnp.array(mc.state_values))

    return Model(c, α, β, ρ, ν, θ, w_grid, z_draws)

Solve the model to compute the reservation wage at the default parameters.

Then address the following questions:

1. How does the reservation wage vary with $\theta$?
2. How does the long run unemployment rate vary with $\theta$?

Can you explain your results?
